In [10]:
import pickle

There are three things that I want to check out here. The previous eda files have more or less been playing around 
with less clear purposes. 

1.) How does this data look - after contacting the forest services, I've been told that each row should be a fire
dectected, and there should be four pictures for every long/lat. pair, correspond to 4 pictures in that given day. 
I need to verify that, and if it is true see how far back that works. 

2.) How does the confidence level range across fires in general, and then how about across the 4 pictures per day? Is there a good range of fire confidence level from 10-90%?

3.) How am I going to group these fires from day to day - do I go out 0.01 degrees in each lat/long direction, 0.10 
degrees, etc.? To look into this what I'm going to do is just cycle through a bunch of different values and see how many fires pop up within that many degrees. 

In [31]:
def row_examination(df):
    counts = df.groupby(['LAT', 'LONG', 'year', 'month', 'day']).count()
    print 'Max. number of rows per lat/long coordinate: ', counts.max()[0]
    print 'Min. number of rows per lat/long coordinate: ', counts.min()[0]
    print 'Mean number of rows per lat/long coordinate: ', counts.mean()[0]

In [32]:
def conf_levels_examination(df): 
    print 'Confidence level info: ', df['CONF'].describe()

In [33]:
for year in xrange(2015, 2002, -1): 
    with open('../../../data/pickled_data/MODIS/df_' + str(year) + '.pkl') as f: 
        df = pickle.load(f)
        print 'Year: ', str(year)
        print '-' * 50, '\n'
        row_examination(df)
        conf_levels_examination(df)

Year:  2015
--------------------------------------------------
Max. number of rows per lat/long coordinate:  10
Min. number of rows per lat/long coordinate:  1
Mean number of rows per lat/long coordinate:  2.05243429241
Confidence level info:  count    140171.000000
mean         66.610961
std          22.428298
min           0.000000
25%          52.000000
50%          67.000000
75%          84.000000
max         100.000000
Name: CONF, dtype: float64
Year:  2014
--------------------------------------------------
Max. number of rows per lat/long coordinate:  7
Min. number of rows per lat/long coordinate:  1
Mean number of rows per lat/long coordinate:  2.07766551185
Confidence level info:  count    229688.000000
mean         68.196545
std          21.343116
min           0.000000
25%          55.000000
50%          70.000000
75%          84.000000
max         100.000000
Name: CONF, dtype: float64
Year:  2013
--------------------------------------------------
Max. number of rows per lat/

From the above, I can see that starting in 2009 it looks like the 4 pictures per lat/long coordinates might be true, but before that it doesn't look to be true. So now I need to just pull out some rows and see exactly what is going on. 

In [79]:
def examine_index(df, index): 
    print df.query('LAT == @index[0] & LONG == @index[1] & year == @index[2] & month == @index[3] & day == @index[4]')

In [87]:
def examine_lown_rows(df): 
    fires_counts = df.groupby(['LAT', 'LONG', 'year', 'month', 'day']).count()['AREA']
    for index in fires_counts[fires_counts == True].index[0:10]:
        print '-' * 50
        examine_index(df, index)

In [88]:
for year in xrange(2015, 2014, -1): 
    with open('../../../data/pickled_data/MODIS/df_' + str(year) + '.pkl') as f: 
        df = pickle.load(f)
        examine_lown_rows(df)     

--------------------------------------------------
                        AREA             PERIMETER   FIRE_  FIRE_ID  \
140132  0.0000000000000E+000  0.0000000000000E+000  140133     4345   

                  LAT           LONG  JULIAN   GMT   TEMP  SPIX  TPIX SAT_SRC  \
140132  2.562800E+001  -9.77430E+001     226  1935  333.2   1.3   1.1       A   

        CONF   FRP  year  month  day  
140132    85  23.1  2015      8   14  
--------------------------------------------------
                        AREA             PERIMETER   FIRE_  FIRE_ID  \
140048  0.0000000000000E+000  0.0000000000000E+000  140049    13250   

                  LAT           LONG  JULIAN   GMT   TEMP  SPIX  TPIX SAT_SRC  \
140048  2.597600E+001  -9.83080E+001     228  1920  326.5   2.2   1.4       A   

        CONF   FRP  year  month  day  
140048    73  33.7  2015      8   16  
--------------------------------------------------
                        AREA             PERIMETER   FIRE_  FIRE_ID  \
140047  